Make sure that these python modules are installed
* pip install pandas
* pip install openpyxl
* pip install scikit-learn

In [19]:
import pandas as pd

file_path = "dataset.xlsx"
sheet_name = "Data"
df = pd.read_excel(file_path, sheet_name = sheet_name)

print(df.head())
print(df.info())

   X0     X1     X2  X3  X4   X5   y
0   1  5.768  62.55   0  28  2.6  28
1   1  8.949  66.91   0  27 -2.0  46
2   1  1.024  34.42   0  20 -3.4  14
3   1  4.534  47.93   0  10  1.1  17
4   1  8.641  35.26   0  30  3.8  32
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X0      50 non-null     int64  
 1   X1      50 non-null     float64
 2   X2      50 non-null     float64
 3   X3      50 non-null     int64  
 4   X4      50 non-null     int64  
 5   X5      50 non-null     float64
 6   y       50 non-null     int64  
dtypes: float64(3), int64(4)
memory usage: 2.9 KB
None


Newton Method

Perform this matrix computation -> ((X'X)^-1)*X'y, use only X and y values from rows 2 to 41

In [20]:
import pandas as pd
import numpy as np

# Selecting rows 2 to 41 for X and y
X_subset = df.iloc[0:40, :-1].values  # Features (all columns except the last)
y_subset = df.iloc[0:40, -1].values  # Target variable (the last column)

"""
# Verifying the dataset
print("X_subset:", X_subset[:5]) 
print("y_subset:", y_subset[:5]) 
print("X_subset:", X_subset[-5:])  
print("y_subset:", y_subset[-5:]) 
"""

# Compute X'X
X_transpose = np.transpose(X_subset)
XtX = np.dot(X_transpose, X_subset)

# Compute (X'X)^-1
XtX_inv = np.linalg.inv(XtX)

# Compute X'y
Xty = np.dot(X_transpose, y_subset)

# Perform the final matrix computation ((X'X)^-1 * X'y)
result = np.dot(XtX_inv, Xty)

# Display the result
result

array([-7.85296896,  3.26574948,  0.16652681, 12.19344681,  0.38258415,
       -1.58862372])

Error Computation (both Training and Test Error)

Generate y values from the new generated w values
Compute the SSR, MSR and RMSE between the generated y values and the actual values both for the data from rows 2 to 41 and the data from rows 43 to 51

In [21]:
# Selecting rows 2 to 41 for X and y
X_subset = df.iloc[0:40, :-1].values  # Features (all columns except the last)
y_subset = df.iloc[0:40, -1].values  # Target variable (the last column)

# Selecting rows 43 to 51 for test data
X_test = df.iloc[41:50, :-1].values  # Features (all columns except the last)
y_test = df.iloc[41:50, -1].values  # Target variable (the last column)

# Select calculated w values from Question #1
w = result

# Generate predicted y values from w
y_train_pred = np.dot(X_subset, w)

# Compute SSR, MSR, and RMSE for the training data
ssr_train = np.sum((y_subset - y_train_pred) ** 2)
msr_train = ssr_train / len(y_subset)
rmse_train = np.sqrt(msr_train)

# Print the Training Errors
print(f"""
Training Errors:
----------------
SSR  : {ssr_train:.4f}
MSR  : {msr_train:.4f}
RMSE : {rmse_train:.4f}
""")

# Generate predicted y values for test data
y_test_pred = np.dot(X_test, w)

# Compute SSR, MSR, and RMSE for the test data
ssr_test = np.sum((y_test - y_test_pred) ** 2)
msr_test = ssr_test / len(y_test)
rmse_test = np.sqrt(msr_test)

# Print the Test Errors
print(f"""
Test Errors:
----------------
SSR  : {ssr_test:.4f}
MSR  : {msr_test:.4f}
RMSE : {rmse_test:.4f}
""")


Training Errors:
----------------
SSR  : 2.9035
MSR  : 0.0726
RMSE : 0.2694


Test Errors:
----------------
SSR  : 0.9341
MSR  : 0.1038
RMSE : 0.3222



Regularized Newton

Next step perform the matrix computation -> ((X'X+lambda*I)^-1)*X'y, using only X and y values from rows 2 to 41
Use lambda as 0.17
Compute and report both training and test errors for the regularized newton computation

In [22]:
from sklearn.linear_model import Ridge  # Import Ridge from sklearn.linear_model
from sklearn.metrics import mean_squared_error, r2_score  # Import evaluation metrics

#Split the data Rows 2 to 11 for training, 12 to 51 for testing
training_data = df.iloc[0:10] 
test_data = df.iloc[10:51]

# Features (X) and target (y) 
X_train = training_data[['X0', 'X1', 'X2', 'X3', 'X4', 'X5']]
y_train = training_data['y']

X_test = test_data[['X0','X1' ,'X2', 'X3', 'X4', 'X5']]
y_test = test_data['y']

# Train the Ridge Regression model with lambda = 0.17
lambda_value = 0.17
model = Ridge(alpha=lambda_value)
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluate the model
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# Calculate R-squared (R²) for training and testing sets
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Return the results: MSE on train, MSE on test, R² on train, and R² on test
print("MSE on train: \n", mse_train)
print("MSE on test: \n", mse_test)
print("R² on train: \n", r2_train)
print("R² on test: \n", r2_test)

MSE on train: 
 0.020556867014720308
MSE on test: 
 33.35916655896695
R² on train: 
 0.9997957588970221
R² on test: 
 0.7401415464810096


Compute R-squared values for both Newton and Regularized Newton on both train and test datasets

Look up how to compute R-squared values in excel

In [23]:
# Selecting rows 2 to 41 for training data (replace with the actual column names or indexes)
X_train = df.iloc[0:40, :-1].values  # Features (all columns except the last)
y_train = df.iloc[0:40, :-1].values  # Target variable (the last column)

# Selecting rows 43 to 51 for test data
X_test = df.iloc[41:50, :-1].values  # Features (all columns except the last)
y_test = df.iloc[41:50, :-1].values  # Target variable (the last column)

# Regularized Newton method
lambda_value = 0.17
num_features = X_train.shape[1]

# Compute X'X
XtX = np.dot(X_train.T, X_train)

# Add regularization term (lambda * I)
regularization_term = lambda_value * np.eye(num_features)
XtX_regularized = XtX + regularization_term

# Compute inverse of (X'X + lambda * I)
XtX_regularized_inv = np.linalg.inv(XtX_regularized)

# Compute X'y
Xty = np.dot(X_train.T, y_train)

# Perform the regularized Newton computation
w_regularized = np.dot(XtX_regularized_inv, Xty)

# Compute predicted y values for training data
y_pred_train_regularized = np.dot(X_train, w_regularized)

# Compute SSR, MSR, and RMSE for the training data (Regularized)
ssr_train_regularized = np.sum((y_train - y_pred_train_regularized) ** 2)
msr_train_regularized = ssr_train_regularized / len(y_train)
rmse_train_regularized = np.sqrt(msr_train_regularized)

print(f"""
Regularized Newton Training Errors:
----------------
SSR  : {ssr_train_regularized:.4f}
MSR  : {msr_train_regularized:.4f}
RMSE : {rmse_train_regularized:.4f}
""")

# Compute predicted y values for test data
y_pred_test_regularized = np.dot(X_test, w_regularized)

# Compute SSR, MSR, and RMSE for the test data (Regularized)
ssr_test_regularized = np.sum((y_test - y_pred_test_regularized) ** 2)
msr_test_regularized = ssr_test_regularized / len(y_test)
rmse_test_regularized = np.sqrt(msr_test_regularized)

print(f"""
Regularized Newton Test Errors:
----------------
SSR  : {ssr_test_regularized:.4f}
MSR  : {msr_test_regularized:.4f}
RMSE : {msr_test_regularized:.4f}
""")


Regularized Newton Training Errors:
----------------
SSR  : 0.0238
MSR  : 0.0006
RMSE : 0.0244


Regularized Newton Test Errors:
----------------
SSR  : 0.0087
MSR  : 0.0010
RMSE : 0.0010



Let's try some more things

For each of the below report the following, MSR on train, MSR on test, R-squared on train and R-squared on test
* Use only rows 2 to 41 for training and rows 42 to 51 for testing and lambda as 0.83


In [24]:
# Selecting rows 2 to 41 for training data
X_train = df.iloc[0:40, :-1].values
y_train = df.iloc[0:40, -1].values

# Selecting rows 42 to 51 for testing data
X_test = df.iloc[40:50, :-1].values
y_test = df.iloc[40:50, -1].values

# Regularized Newton method
lambda_value = 0.83
num_features = X_train.shape[1]

# Compute X'X
XtX = np.dot(X_train.T, X_train)

# Add regularization term (lambda * I)
regularization_term = lambda_value * np.eye(num_features)
XtX_regularized = XtX + regularization_term

# Compute inverse of (X'X + lambda * I)
XtX_regularized_inv = np.linalg.inv(XtX_regularized)

# Compute X'y
Xty = np.dot(X_train.T, y_train)

# Perform the regularized Newton computation
w_regularized = np.dot(XtX_regularized_inv, Xty)

# Compute predicted y values for training data
y_pred_train_regularized = np.dot(X_train, w_regularized)

# Compute predicted y values for test data
y_pred_test_regularized = np.dot(X_test, w_regularized)

# Compute MSR for training data
msr_train = mean_squared_error(y_train, y_pred_train_regularized)
# Compute MSR for test data
msr_test = mean_squared_error(y_test, y_pred_test_regularized)

# Compute R-squared for training and test data
r2_train = r2_score(y_train, y_pred_train_regularized)
r2_test = r2_score(y_test, y_pred_test_regularized)

# Return the results: MSE on train, MSE on test, R² on train, and R² on test
print(f"MSR on Training Data: {msr_train:.4f}")
print(f"MSR on Test Data: {msr_test:.4f}")
print(f"R-squared on Training Data: {r2_train:.4f}")
print(f"R-squared on Test Data: {r2_test:.4f}")

MSR on Training Data: 0.8328
MSR on Test Data: 1.2122
R-squared on Training Data: 0.9895
R-squared on Test Data: 0.9956


* Use only rows 2 to 26 for training and rows 27 to 51 for testing and lambda as 0.17

In [25]:
# Selecting rows 2 to 26 for training data
X_train = df.iloc[0:25, :-1].values
y_train = df.iloc[0:25, -1].values

# Selecting rows 27 to 51 for testing data
X_test = df.iloc[25:50, :-1].values
y_test = df.iloc[25:50, -1].values

# Regularized Newton method
lambda_value = 0.17
num_features = X_train.shape[1]

# Compute X'X
XtX = np.dot(X_train.T, X_train)

# Add regularization term (lambda * I)
regularization_term = lambda_value * np.eye(num_features)
XtX_regularized = XtX + regularization_term

# Compute inverse of (X'X + lambda * I)
XtX_regularized_inv = np.linalg.inv(XtX_regularized)

# Compute X'y
Xty = np.dot(X_train.T, y_train)

# Perform the regularized Newton computation
w_regularized = np.dot(XtX_regularized_inv, Xty)

# Compute predicted y values for training data
y_pred_train_regularized = np.dot(X_train, w_regularized)

# Compute predicted y values for test data
y_pred_test_regularized = np.dot(X_test, w_regularized)

# Compute MSR for training data
msr_train = mean_squared_error(y_train, y_pred_train_regularized)
# Compute MSR for test data
msr_test = mean_squared_error(y_test, y_pred_test_regularized)

# Compute R-squared for training and test data
r2_train = r2_score(y_train, y_pred_train_regularized)
r2_test = r2_score(y_test, y_pred_test_regularized)

# Return the results: MSE on train, MSE on test, R² on train, and R² on test
print(f"MSR on Training Data: {msr_train:.4f}")
print(f"MSR on Test Data: {msr_test:.4f}")
print(f"R-squared on Training Data: {r2_train:.4f}")
print(f"R-squared on Test Data: {r2_test:.4f}")

MSR on Training Data: 0.2136
MSR on Test Data: 0.2071
R-squared on Training Data: 0.9979
R-squared on Test Data: 0.9986


* Use only rows 2 to 26 for training and rows 27 to 51 for testing and lambda as 0.83

In [26]:
# Selecting rows 2 to 26 for training data
X_train = df.iloc[0:25, :-1].values
y_train = df.iloc[0:25, -1].values

# Selecting rows 27 to 51 for testing data
X_test = df.iloc[25:50, :-1].values
y_test = df.iloc[25:50, -1].values

# Regularized Newton method
lambda_value = 0.83
num_features = X_train.shape[1]

# Compute X'X
XtX = np.dot(X_train.T, X_train)

# Add regularization term (lambda * I)
regularization_term = lambda_value * np.eye(num_features)
XtX_regularized = XtX + regularization_term

# Compute inverse of (X'X + lambda * I)
XtX_regularized_inv = np.linalg.inv(XtX_regularized)

# Compute X'y
Xty = np.dot(X_train.T, y_train)

# Perform the regularized Newton computation
w_regularized = np.dot(XtX_regularized_inv, Xty)

# Compute predicted y values for training data
y_pred_train_regularized = np.dot(X_train, w_regularized)

# Compute predicted y values for test data
y_pred_test_regularized = np.dot(X_test, w_regularized)

# Compute MSR for training data
msr_train = mean_squared_error(y_train, y_pred_train_regularized)
# Compute MSR for test data
msr_test = mean_squared_error(y_test, y_pred_test_regularized)

# Compute R-squared for training and test data
r2_train = r2_score(y_train, y_pred_train_regularized)
r2_test = r2_score(y_test, y_pred_test_regularized)

# Return the results: MSE on train, MSE on test, R² on train, and R² on test
print(f"MSR on Training Data: {msr_train:.4f}")
print(f"MSR on Test Data: {msr_test:.4f}")
print(f"R-squared on Training Data: {r2_train:.4f}")
print(f"R-squared on Test Data: {r2_test:.4f}")

MSR on Training Data: 1.5532
MSR on Test Data: 1.9270
R-squared on Training Data: 0.9847
R-squared on Test Data: 0.9866


* Use only rows 2 to 11 for training and rows 12 to 51 for testing and lambda as 0.17

In [27]:
# Selecting rows 2 to 11 for training data
X_train = df.iloc[0:10, :-1].values
y_train = df.iloc[0:10, -1].values

# Selecting rows 12 to 51 for testing data
X_test = df.iloc[10:50, :-1].values
y_test = df.iloc[10:50, -1].values

# Regularized Newton method
lambda_value = 0.17
num_features = X_train.shape[1]

# Compute X'X
XtX = np.dot(X_train.T, X_train)

# Add regularization term (lambda * I)
regularization_term = lambda_value * np.eye(num_features)
XtX_regularized = XtX + regularization_term

# Compute inverse of (X'X + lambda * I)
XtX_regularized_inv = np.linalg.inv(XtX_regularized)

# Compute X'y
Xty = np.dot(X_train.T, y_train)

# Perform the regularized Newton computation
w_regularized = np.dot(XtX_regularized_inv, Xty)

# Compute predicted y values for training data
y_pred_train_regularized = np.dot(X_train, w_regularized)

# Compute predicted y values for test data
y_pred_test_regularized = np.dot(X_test, w_regularized)

# Compute MSR for training data
msr_train = mean_squared_error(y_train, y_pred_train_regularized)
# Compute MSR for test data
msr_test = mean_squared_error(y_test, y_pred_test_regularized)

# Compute R-squared for training and test data
r2_train = r2_score(y_train, y_pred_train_regularized)
r2_test = r2_score(y_test, y_pred_test_regularized)

# Return the results: MSE on train, MSE on test, R² on train, and R² on test
print(f"MSR on Training Data: {msr_train:.4f}")
print(f"MSR on Test Data: {msr_test:.4f}")
print(f"R-squared on Training Data: {r2_train:.4f}")
print(f"R-squared on Test Data: {r2_test:.4f}")

MSR on Training Data: 0.2538
MSR on Test Data: 32.3167
R-squared on Training Data: 0.9975
R-squared on Test Data: 0.7483


* Use only rows 2 to 11 for training and rows 12 to 51 for testing and lambda as 0.83

In [28]:
# Selecting rows 2 to 11 for training data
X_train = df.iloc[0:10, :-1].values
y_train = df.iloc[0:10, -1].values

# Selecting rows 12 to 51 for testing data
X_test = df.iloc[10:50, :-1].values
y_test = df.iloc[10:50, -1].values

# Regularized Newton method
lambda_value = 0.83
num_features = X_train.shape[1]

# Compute X'X
XtX = np.dot(X_train.T, X_train)

# Add regularization term (lambda * I)
regularization_term = lambda_value * np.eye(num_features)
XtX_regularized = XtX + regularization_term

# Compute inverse of (X'X + lambda * I)
XtX_regularized_inv = np.linalg.inv(XtX_regularized)

# Compute X'y
Xty = np.dot(X_train.T, y_train)

# Perform the regularized Newton computation
w_regularized = np.dot(XtX_regularized_inv, Xty)

# Compute predicted y values for training data
y_pred_train_regularized = np.dot(X_train, w_regularized)

# Compute predicted y values for test data
y_pred_test_regularized = np.dot(X_test, w_regularized)

# Compute MSR for training data
msr_train = mean_squared_error(y_train, y_pred_train_regularized)
# Compute MSR for test data
msr_test = mean_squared_error(y_test, y_pred_test_regularized)

# Compute R-squared for training and test data
r2_train = r2_score(y_train, y_pred_train_regularized)
r2_test = r2_score(y_test, y_pred_test_regularized)

# Return the results: MSE on train, MSE on test, R² on train, and R² on test
print(f"MSR on Training Data: {msr_train:.4f}")
print(f"MSR on Test Data: {msr_test:.4f}")
print(f"R-squared on Training Data: {r2_train:.4f}")
print(f"R-squared on Test Data: {r2_test:.4f}")

MSR on Training Data: 1.2653
MSR on Test Data: 31.4747
R-squared on Training Data: 0.9874
R-squared on Test Data: 0.7548
